# Cadastro Geral

Atualmente há duas lista de cadastro geral no Ponto do Bem. O objetivo desse programa é conciliar os dados das duas em uma planilha só. Ademais, userei VBA para corresponder dados novos em futuras atualizaçôes.



In [1]:
import pandas as pd
import numpy as np

In [91]:
geral_oficial = pd.read_csv("/content/CADASTRO_GERAL.csv", parse_dates=True)
geral_nao_oficial = pd.read_csv("/content/Cadastro_Geral_Nao_oficial.csv", parse_dates=True)
geral_maio = pd.read_excel("Cadastro-Geral-ATUALIZADO-MAIO-2021.xlsx", parse_dates=True)

## Visualizações

In [3]:
geral_nao_oficial.tail()

,Nº,Nome,Liberado?,Remédios / Roupas,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Tipo de Auxílio,Qtde de pessoas,(R$),Data de Atualização,Endereço,CPF,DATA DA PLANILHA,Observações
1363,1357.0,Zilda Maria Dias Soares (amiga da Eliane),NaN,NaN,Praça Seca,99551-2776,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,4/23/2020,NaN
1364,1358.0,ZILDA SOARES,NaN,NaN,Praça Seca,99551-2776,2,2,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,5/8/2020,NaN
1365,1359.0,ZILMA DA SILVA MAIA,NaN,NaN,Praça Seca,99428-8934/97397-3521 (vizinha),NaN,2,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,5/8/2020,NaN
1366,1360.0,ZULEICA SILVA DO JANUÁRIO,NaN,NaN,NaN,99193-5560,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,5/22/2020,NaN
1367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
geral_oficial.tail()

,Nº+,Nome,EV/AB/PRAÇA,Pedidos de Roupa/Remédio,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Recebe Auxílio?,Qtde de pessoas,Renda mensal,Data de entrada,Endereço,CPF,CESTA 1,CESTA 2,CESTA 3,Última atualização,Observações
1536,1537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1537,1538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1538,1539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1539,1540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1540,1541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vemos que a lista oficial tem mais dados do que a não oficial.

In [5]:
geral_nao_oficial.shape, geral_oficial.shape

((1368, 18), (1541, 21))

In [6]:
geral_oficial.dtypes

Nº+                          object
Nome                         object
EV/AB/PRAÇA                  object
Pedidos de Roupa/Remédio     object
Bairro                       object
Telefone                     object
Crianças                     object
Adultos                      object
Idosos                       object
Trabalha                     object
Recebe Auxílio?              object
Qtde de pessoas              object
Renda mensal                 object
Data de entrada              object
Endereço                     object
CPF                          object
CESTA 1                      object
CESTA 2                      object
CESTA 3                     float64
Última atualização           object
Observações                  object
dtype: object

A lista oficial foi toda reconhecida como string, exceto a coluna da Cesta 3. 

In [7]:
geral_nao_oficial.dtypes

Nº                     float64
Nome                    object
Liberado?               object
Remédios / Roupas       object
Bairro                  object
Telefone                object
Crianças                object
Adultos                 object
Idosos                  object
Trabalha                object
Tipo de Auxílio         object
Qtde de pessoas         object
(R$)                    object
Data de Atualização     object
Endereço                object
CPF                     object
DATA DA PLANILHA        object
Observações             object
dtype: object

## Modificando e validando dados

### Excluindo e Renomeando colunas

In [92]:
geral_nao_oficial = geral_nao_oficial.drop("Liberado?", axis = 1) # Exluiu a coluna Liberado?
geral_nao_oficial = geral_nao_oficial.drop("Nº", axis = 1) # Exclui a coluna Nº

Para melhor usar o join/merge/combine faremos as colunas nas duas planilhas terem o mesmo nome quando necessárias.

In [93]:
nomes = {"Remédios / Roupas": "Pedidos de Roupa/Remédio", "Tipo de Auxílio": "Recebe Auxílio?",
         "(R$)": "Renda mensal", "DATA DA PLANILHA": "Data de entrada", "Data de Atualização": "Última atualização"}

In [94]:
geral_nao_oficial.rename(nomes, axis = 1, inplace=True)

In [95]:
geral_nao_oficial.head()

,Nome,Pedidos de Roupa/Remédio,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Recebe Auxílio?,Qtde de pessoas,Renda mensal,Última atualização,Endereço,CPF,Data de entrada,Observações
0,Abdaline Jean Baptiste,NaN,Tanque,98064-1512 - 97201-0948,3,2,2,não,NaN,7,NaN,6/19/2020,"Rua Almirante Melquíades de Souza, 322 Tanque ...",065.001.667-01,7/3/2020,NaN
1,Abigail Pereira Cabral,NaN,Praça Seca,998039-7091 (tel errado),1,3,NaN,não,NaN,4,NaN,6/5/2020,NaN,714.666.306-34,7/3/2020,NaN
2,Adélia Verônica Pinto da Silva,Sim,Vila Isabel,99312-1634,1,2,0,não,"bolsa famila 130,00",3,NaN,4/20/2021,Rua viana drumond 38 casa 2 Vila Isabel,125.458..797-70,7/11/2020,NaN
3,Adelino da Silva Gomes,NaN,Vila Isabel,98891-2516,3,1,NaN,NaN,NaN,4,NaN,NaN,"Rua Armando de Albuquerque, 33",011.833.607-03,6/27/2020,NaN
4,Aderaldo Araújo Simplício,NaN,Tijuca,99947-3523,NaN,NaN,NaN,NaN,NaN,0,NaN,7/25/2020,NaN,606.569.647-15,8/8/2020,NaN


In [96]:
geral_oficial.rename({"Nº+": "Nº"}, axis = 1, inplace= True)

### Estruturando a Coluna "CPF"

In [97]:
# Essa função enviar cada valor para ser checado
def separacao(data_serie):
  num = 0

  for i in range(len(data_serie)):
    try:
      if math.isnan(data_serie[i]):
        # Se o valor for NaN ele pula
        continue
        
    except TypeError:
        pass

    if type(data_serie[i]) is list:
      # Se for uma lista, junta os valores ao converte-los para string
      data_serie[i] = [str(x) for x in data_serie[i]]
      data_serie[i] = "".join(data_serie[i])

    if data_serie[i].startswith("ID:") or data_serie[i].startswith("RG") or data_serie[i].startswith("Ca"):
      # Se começa com "ID:" ou "RG" é ignorado
      continue

    if data_serie[i].startswith("?"):
      # se o valor for "?" é trocado por NaN
      data_serie[i] = np.nan
      continue

    valores = cpf(data_serie[i])
    num += 1

    if num % 10 == 0:
      # Printa a cada 10 CPF
      print(data_serie[i], valores[0], "index =", i)

    if valores[1] == False:
      data_serie[i] = valores[0]
  
  return data_serie


In [98]:
import math
# Essa função checará se o CPF está no formato XXX.XXX.XXX-XX
def cpf(valor):
  
  cpf = []
  valor = list(valor.strip()) # Transforma em uma lista
      
  for i in range(len(valor)):
    # Começa a verificar do ultimo ao primeiro
        
    if valor[i].isdigit():
      # Se o valor for digito

      cpf.append(str(valor[i]))
    
  while len(cpf) < 11:
    # se a lista for menor que 11 será adicionado um 0 até completar 11 digito

    cpf.insert(0, "0")
  
  # Insere os caracteres nas posições específicas
  cpf.insert(3, ".") 
  cpf.insert(7, ".")
  cpf.insert(11, "-")

  return ["".join(cpf), False]


    


In [99]:
# Passando a serie da Coluna CPF
coluna_cpf = geral_nao_oficial.loc[:, ("CPF")]
coluna_cpf = separacao(coluna_cpf)

127.383.047-45 127.383.047-45 index = 13
002.106.317-67 002.106.317-67 index = 25
014.187.861-44 014.187.861-44 index = 46
171.637.447-01 171.637.447-01 index = 67
035.672.627-42 035.672.627-42 index = 101
025.869.567-61 025.869.567-61 index = 126
917.828.487-20 917.828.487-20 index = 145
884.896.713-00 884.896.713-00 index = 165
149.072.277-79 149.072.277-79 index = 190
098.072.117-27 098.072.117-27 index = 219
144.283.937-94 144.283.937-94 index = 240
821.274.767-00 821.274.767-00 index = 261
109.594.397-90 109.594.397-90 index = 289
185.644.247-03 185.644.247-03 index = 313
125.674.487-56 125.674.487-56 index = 327
108.692.427-41 108.692.427-41 index = 347
025.896.447-28 025.896.447-28 index = 370
13197472721 131.974.727-21 index = 398
035.126.327-69 035.126.327-69 index = 419
103.416.957-26 103.416.957-26 index = 438
088.225.977-61 088.225.977-61 index = 456
079.587.877-07 079.587.877-07 index = 475
930.00 000.000.930-00 index = 498
15128200752 151.282.007-52 index = 518
158.578.39

In [100]:
coluna_cpf_oficial = separacao(geral_oficial.loc[:, ("CPF")])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


150.421.007-76 150.421.007-76 index = 9
151.150.397-10 151.150.397-10 index = 20
769.020.057-53 769.020.057-53 index = 33
103.280.958-45 103.280.958-45 index = 49
154.836.677-35 154.836.677-35 index = 66
118.869.577-03 118.869.577-03 index = 85
035.672.627-42 035.672.627-42 index = 111
077.616.937-81 077.616.937-81 index = 139
820.732.793-68 820.732.793-68 index = 156
072.311.583-40 072.311.583-40 index = 176
149.072.277.79 149.072.277-79 index = 200
159.617.717-92 159.617.717-92 index = 218
112.771.037-04 112.771.037-04 index = 245
164.085.237-90 164.085.237-90 index = 267
782.377.507-72 782.377.507-72 index = 282
 099.314.027-09 099.314.027-09 index = 306
159.938.247-43 159.938.247-43 index = 321
185.644.247-03 185.644.247-03 index = 339
125.674.487-56 125.674.487-56 index = 354
722.627.887-15 722.627.887-15 index = 373
132.215.257-88 132.215.257-88 index = 391
180.230.717-69 180.230.717-69 index = 420
091.891.747-63 091.891.747-63 index = 438
744.426.197-00 744.426.197-00 index = 46

Agora que padronizamos o CPF, iremos colocar no DataFrame.

In [101]:
geral_nao_oficial["CPF"] = coluna_cpf
geral_oficial["CPF"] = coluna_cpf_oficial

In [102]:
geral_nao_oficial["CPF"].head()

0    065.001.667-01
1    714.666.306-34
2    125.458.797-70
3    011.833.607-03
4    606.569.647-15
Name: CPF, dtype: object

### Tirando todos os dados "?????"

In [103]:
lista = ["?", "??", "???", "????", "?????"]

In [104]:
geral_nao_oficial.replace(lista, np.nan, inplace= True)
geral_oficial.replace(lista, np.nan, inplace= True)

### Excluindo linhas vazias

In [105]:
def exclui_linha_vazia(tabela):

  for i in range(tabela.shape[0]):

    if pd.isnull(tabela['Nome'][i]):
      tabela.drop([i], inplace = True)
  
  return tabela

In [106]:
geral_oficial = exclui_linha_vazia(geral_oficial)
geral_nao_oficial = exclui_linha_vazia(geral_nao_oficial)
geral_maio = exclui_linha_vazia(geral_maio)

## Juntando as duas planilhas

In [107]:
Colunas = ["CPF", "Nome", "Telefone", "Data de entrada"]

In [108]:
mergido = pd.merge(geral_nao_oficial, geral_oficial, on = Colunas, how="outer")

In [109]:
geral_nao_oficial.shape, geral_oficial.shape, mergido.shape

((1367, 16), (1506, 21), (2813, 33))

In [110]:
mergido.tail()

,Nome,Pedidos de Roupa/Remédio_x,Bairro_x,Telefone,Crianças_x,Adultos_x,Idosos_x,Trabalha_x,Recebe Auxílio?_x,Qtde de pessoas_x,Renda mensal_x,Última atualização_x,Endereço_x,CPF,Data de entrada,Observações_x,Nº,EV/AB/PRAÇA,Pedidos de Roupa/Remédio_y,Bairro_y,Crianças_y,Adultos_y,Idosos_y,Trabalha_y,Recebe Auxílio?_y,Qtde de pessoas_y,Renda mensal_y,Endereço_y,CESTA 1,CESTA 2,CESTA 3,Última atualização_y,Observações_y
2808,ZILDA SOARES,NaN,NaN,99551-2776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1502,NaN,NaN,Praça Seca,2,2,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,08/05/2020,NaN
2809,ZILMA DA SILVA MAIA,NaN,NaN,99428-8934/97397-3521 (vizinha),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1503,NaN,NaN,Praça Seca,NaN,2,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,08/05/2020,NaN
2810,ZULEICA SILVA DO JANUÁRIO,NaN,NaN,99193-5560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,22/05/2020,NaN
2811,LORREYNE FARIA DA SILVA,NaN,NaN,97448-5093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163.294.047-71,19/05/2021,NaN,1505,AB,NaN,Tijuca,2,2,0,não,não,4,"375,00","Rua Dona Sinha, 1 - casa 8",NaN,NaN,NaN,20/05/2021,NaN
2812,RELAYNE DA CONCEIÇÃO DUARTE,NaN,NaN,96657-5110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19/05/2021,NaN,#REF!,AB,NaN,Tijuca,5,1,1,não,Bolsa familia,7,"405,00","Rua Joaquim Pizarro, 18",NaN,NaN,NaN,20/05/2021,NaN


In [111]:
combinado = geral_oficial.combine_first(geral_nao_oficial)

In [112]:
combinado.shape

(1506, 21)

In [113]:
combinado.head()

,Adultos,Bairro,CESTA 1,CESTA 2,CESTA 3,CPF,Crianças,Data de entrada,EV/AB/PRAÇA,Endereço,Idosos,Nome,Nº,Observações,Pedidos de Roupa/Remédio,Qtde de pessoas,Recebe Auxílio?,Renda mensal,Telefone,Trabalha,Última atualização
0,1,Tanque,NaN,NaN,NaN,065.001.667-01,4,7/3/2020,Praça,Rua Almirante de Souza - Casa 322,2,Abdaline Jean Baptiste,1,NaN,Não,7,Não,"1.100,00",98064-1512,Não,22/03/2021
1,3,Praça Seca,NaN,NaN,NaN,714.666.306-34,1,05/06/2020,NaN,NaN,NaN,Abigail Pereira Cabral,2,NaN,NaN,4,NaN,NaN,998039-7091 (tel errado),não,03/07/2020
2,1,Vila Isabel,NaN,NaN,NaN,125.458.797-70,1,11/08/2020,AB,"Rua Senador Nabuco, 411",1,Adélia Verônica Pinto da Silva,3,NaN,Sim,3,Não,"0,00",99312-1634,Não,11/07/2020
3,2,Vila Isabel,NaN,NaN,NaN,011.833.607-03,2,29/05/2020,Praça,"Rua Armando de Albuquerque, 33",0,Adelino da Silva Gomes,4,NaN,Sim,4,Não,"0,00",(21) 98891-2516,Não,29/05/2020
4,NaN,Tijuca,NaN,NaN,NaN,606.569.647-15,NaN,25/07/2020,NaN,NaN,NaN,Aderaldo Araújo Simplício,5,NaN,NaN,0,NaN,NaN,99947-3523,NaN,08/08/2020


### Reordenando as colunas

In [114]:
cols = list(geral_oficial.columns.values)
cols

['Nº',
 'Nome',
 'EV/AB/PRAÇA',
 'Pedidos de Roupa/Remédio',
 'Bairro',
 'Telefone',
 'Crianças',
 'Adultos',
 'Idosos',
 'Trabalha',
 'Recebe Auxílio?',
 'Qtde de pessoas',
 'Renda mensal',
 'Data de entrada',
 'Endereço',
 'CPF',
 'CESTA 1',
 'CESTA 2',
 'CESTA 3',
 'Última atualização',
 'Observações']

In [115]:
combinado = combinado[cols]
combinado

,Nº,Nome,EV/AB/PRAÇA,Pedidos de Roupa/Remédio,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Recebe Auxílio?,Qtde de pessoas,Renda mensal,Data de entrada,Endereço,CPF,CESTA 1,CESTA 2,CESTA 3,Última atualização,Observações
0,1,Abdaline Jean Baptiste,Praça,Não,Tanque,98064-1512,4,1,2,Não,Não,7,"1.100,00",7/3/2020,Rua Almirante de Souza - Casa 322,065.001.667-01,NaN,NaN,NaN,22/03/2021,NaN
1,2,Abigail Pereira Cabral,NaN,NaN,Praça Seca,998039-7091 (tel errado),1,3,NaN,não,NaN,4,NaN,05/06/2020,NaN,714.666.306-34,NaN,NaN,NaN,03/07/2020,NaN
2,3,Adélia Verônica Pinto da Silva,AB,Sim,Vila Isabel,99312-1634,1,1,1,Não,Não,3,"0,00",11/08/2020,"Rua Senador Nabuco, 411",125.458.797-70,NaN,NaN,NaN,11/07/2020,NaN
3,4,Adelino da Silva Gomes,Praça,Sim,Vila Isabel,(21) 98891-2516,2,2,0,Não,Não,4,"0,00",29/05/2020,"Rua Armando de Albuquerque, 33",011.833.607-03,NaN,NaN,NaN,29/05/2020,NaN
4,5,Aderaldo Araújo Simplício,NaN,NaN,Tijuca,99947-3523,NaN,NaN,NaN,NaN,NaN,0,NaN,25/07/2020,NaN,606.569.647-15,NaN,NaN,NaN,08/08/2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501,1502,ZILDA SOARES,NaN,NaN,Praça Seca,99551-2776,2,2,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/05/2020,NaN
1502,1503,ZILMA DA SILVA MAIA,NaN,NaN,Praça Seca,99428-8934/97397-3521 (vizinha),NaN,2,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/05/2020,NaN
1503,1504,ZULEICA SILVA DO JANUÁRIO,NaN,NaN,NaN,99193-5560,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020,NaN
1504,1505,LORREYNE FARIA DA SILVA,AB,NaN,Tijuca,97448-5093,2,2,0,não,não,4,"375,00",19/05/2021,"Rua Dona Sinha, 1 - casa 8",163.294.047-71,NaN,NaN,NaN,20/05/2021,NaN


### Atualizando a lista combinada

In [116]:
# Essa função passa os dados para checagem
def checa(data_frame):

  for i in range(data_frame.shape[0]):

    if data_frame["EV/AB/PRAÇA"][i] in ["Praça", "praça", "Praca", "praca", "PRAÇA", "PRACA"]:
      # Se a pessoa é da Praça checa o valor na coluna Cesta a

      try:
        if math.isnan(data_frame["CESTA 1"][i]):
          # Se a Cesta 1 for NaN 

          try:
            if math.isnan(data_frame["Data de entrada"][i]):
              # Checa se a data de entrada é nula
              continue

          except TypeError:
            data_frame["CESTA 1"][i] = data_frame["Data de entrada"][i]

      except TypeError:
        continue

  return data_frame


In [117]:
geral_oficial = checa(combinado)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [118]:
geral_oficial

,Nº,Nome,EV/AB/PRAÇA,Pedidos de Roupa/Remédio,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Recebe Auxílio?,Qtde de pessoas,Renda mensal,Data de entrada,Endereço,CPF,CESTA 1,CESTA 2,CESTA 3,Última atualização,Observações
0,1,Abdaline Jean Baptiste,Praça,Não,Tanque,98064-1512,4,1,2,Não,Não,7,"1.100,00",7/3/2020,Rua Almirante de Souza - Casa 322,065.001.667-01,7/3/2020,NaN,NaN,22/03/2021,NaN
1,2,Abigail Pereira Cabral,NaN,NaN,Praça Seca,998039-7091 (tel errado),1,3,NaN,não,NaN,4,NaN,05/06/2020,NaN,714.666.306-34,NaN,NaN,NaN,03/07/2020,NaN
2,3,Adélia Verônica Pinto da Silva,AB,Sim,Vila Isabel,99312-1634,1,1,1,Não,Não,3,"0,00",11/08/2020,"Rua Senador Nabuco, 411",125.458.797-70,NaN,NaN,NaN,11/07/2020,NaN
3,4,Adelino da Silva Gomes,Praça,Sim,Vila Isabel,(21) 98891-2516,2,2,0,Não,Não,4,"0,00",29/05/2020,"Rua Armando de Albuquerque, 33",011.833.607-03,29/05/2020,NaN,NaN,29/05/2020,NaN
4,5,Aderaldo Araújo Simplício,NaN,NaN,Tijuca,99947-3523,NaN,NaN,NaN,NaN,NaN,0,NaN,25/07/2020,NaN,606.569.647-15,NaN,NaN,NaN,08/08/2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501,1502,ZILDA SOARES,NaN,NaN,Praça Seca,99551-2776,2,2,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/05/2020,NaN
1502,1503,ZILMA DA SILVA MAIA,NaN,NaN,Praça Seca,99428-8934/97397-3521 (vizinha),NaN,2,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/05/2020,NaN
1503,1504,ZULEICA SILVA DO JANUÁRIO,NaN,NaN,NaN,99193-5560,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22/05/2020,NaN
1504,1505,LORREYNE FARIA DA SILVA,AB,NaN,Tijuca,97448-5093,2,2,0,não,não,4,"375,00",19/05/2021,"Rua Dona Sinha, 1 - casa 8",163.294.047-71,NaN,NaN,NaN,20/05/2021,NaN


# Comparando a planilha atual (26/08/2021) com a feita em 05/2021

Devido a algumas incongruências nos dados, foi necessário checar se houve algum erro na combinação das duas planilhas usadas anteriormente. Veremos se a mesma pessoa cadastrada tem os dados compativeis nas duas planilhas

In [134]:
colunas = geral_maio.columns.values
colunas

array(['Nº', 'Nome', 'EV/AB/PRAÇA', 'Pedidos de Roupa/Remédio', 'Bairro',
       'Telefone', 'Crianças', 'Adultos', 'Idosos', 'Trabalha',
       'Recebe Auxílio?', 'Qtde de pessoas', 'Renda mensal',
       'Data de entrada', 'Endereço', 'CPF', 'CESTA 1', 'CESTA 2',
       'CESTA 3', 'Última atualização', 'Observações'], dtype=object)

In [69]:
def encontra_index(tabela):

  index = []

  for i in range(tabela.shape[0]):

    tempo = tabela['Última atualização'][i]
    if (pd.isnull(tempo)): continue

    if (tempo.month >= 5) and (tempo.year == 2021):

      if (tempo.month == 5) and (tempo.day < 22):
        continue
      else:
        index.append(i) 
  
  return index

In [145]:
def dados_atualizados(tabela):

  index = encontra_index(tabela)

  atualizados = pd.DataFrame()

  for i in index:

    atualizados = atualizados.append([tabela.iloc[i, :]])

  return atualizados   

In [188]:
Atualizados = dados_atualizados(geral_maio)

Atualizados.shape

(222, 21)

In [196]:
Atualizados.head()

,Nº,Nome,EV/AB/PRAÇA,Pedidos de Roupa/Remédio,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Recebe Auxílio?,Qtde de pessoas,Renda mensal,Data de entrada,Endereço,CPF,CESTA 1,CESTA 2,CESTA 3,Última atualização,Observações
10,11,Adriana da Silva dos Santos,Praça,NaN,NaN,99907-8115,?,?,?,?,?,?,?,2021-05-08 00:00:00,NaN,121.829.767-01,2021-05-08,NaT,NaT,2021-05-27,NaN
38,39,Alessandra Ferreira Benedito,Praça,NaN,Mangueira,98157-9255 / 98231-9162,3,2,0,Vende Doce,Bolsa Família,5,89,2021-05-26 00:00:00,"Rua Visconde de Niterói, 650 - casa 35",061.579.017-80,2021-08-14,NaT,NaT,2021-08-27,NaN
44,45,Alex Santos de Oliveira,Praça,NaN,Tijuca,982338343,?,?,?,?,?,?,?,2021-05-08 00:00:00,NaN,099.154.227-41,2021-05-08,NaT,NaT,2021-05-27,NaN
45,46,Alexandra da SIlva Barreto,Praça,NaN,Lins,99314-8348,4,3,0,Não,Não,7,0,2021-07-10 00:00:00,NaN,018.314.037-06,2021-08-14,NaT,NaT,2021-08-27,NaN
46,47,Alexandra da Silva Ferreira,Praça,NaN,Vila Isabel,98233-8343,5,1,0,não,Bolsa Familia,6,475,2021-05-08 00:00:00,"Rua Célia Regina, casa 12 b",099.154.227-41,2021-05-08,2021-08-14,NaT,2021-08-27,NaN


Procurando nomes da planilha nova com a antiga. 

In [222]:
def nomes_da_nova(tabela):

  nomes_novo = []
  lista =  tabela['Nº'].values
  x = 0

  for i in tabela['Nome']:

    nomes_novo.append([i, lista[x]-1, '', '', ''])
    x += 1

  return nomes_novo
    

In [227]:
def DataFrameNomes(oficial, oficial1, nao_oficial, nomes):

  for j in range(oficial.shape[0]):

    for k in range(len(nomes)):

      if oficial["Nome"][j] == nomes[k][0]:
        nomes[k][2] = j
        break


  for j in range(oficial1.shape[0]):

    for k in range(len(nomes)):

      if oficial1["Nome"][j] == nomes[k][0]:
        nomes[k][3] = j
        break
  
  for j in range(nao_oficial.shape[0]):

    for k in range(len(nomes)):

      if nao_oficial["Nome"][j] == nomes[k][0]:
        nomes[k][4] = j
        break

  return nomes

In [224]:
geral_oficial1 = pd.read_csv("/content/CADASTRO_GERAL.csv", parse_dates=True)

In [228]:
nomes_novos = nomes_da_nova(Atualizados)
tabela_nomes_index = DataFrameNomes(geral_oficial,geral_oficial1, geral_nao_oficial, nomes_novos) 
relacao_nome = pd.DataFrame(data=tabela_nomes_index, columns=("Nome", "Maio", "oficial", "oficial1", "nao-oficial"))
relacao_nome.head(10)

,Nome,Maio,oficial,oficial1,nao-oficial
0,Adriana da Silva dos Santos,10,,,
1,Alessandra Ferreira Benedito,38,,,
2,Alex Santos de Oliveira,44,41,41,37
3,Alexandra da SIlva Barreto,45,,,
4,Alexandra da Silva Ferreira,46,,,
5,Alexandra Rios,49,44,44,40
6,Alick Josef,52,47,47,43
7,Aline Anjo dos Santos,53,,,
8,Aline Basilio de Souza,54,,,
9,Aline de Souza Caetano,56,,,


In [229]:
geral_maio["Nome"][44], geral_maio["CPF"][44]

('Alex Santos de Oliveira', '099.154.227-41')

In [235]:
geral_oficial["Nome"][44], geral_oficial["CPF"][44]

('Alexandra Rios', '103.280.958-45')

Aparentemente o erro data na planilha 'Geral_oficial'. Resta saber se ele provem da junção da geral_oficial de 22 de maio com a mais antiga.

In [233]:
geral_oficial1['Nome'][44], geral_oficial1["CPF"][44]

('Alexandra Rios', nan)

In [234]:
geral_nao_oficial['Nome'][40], geral_nao_oficial["CPF"][40]

('Alexandra Rios', nan)

**Nas planilhas antigas, "Alex Santos de Oliveira" não tinha o CPF no registro,mas de alguma forma ele tem depois da combinação das duas (geral_oficial)**

# Consertando o erro

# Salvando a planilha

In [ ]:
geral_oficial.to_excel("Cadastro_Geral.xlsx", sheet_name="Cadastro", index=False)